# Cortisol Plots

In [ ]:
from pathlib import Path
import json

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.stats import StatsPipeline
from biopsykit.io import load_long_format_csv
from biopsykit.utils.dataframe_handling import multi_xs

from fau_colors import cmaps

import matplotlib.pyplot as plt
import seaborn as sns

from carwatch_analysis.io import load_cortisol_samples_reporting_times
from carwatch_analysis.utils import describe_groups_df
from carwatch_analysis.stats import create_unique_night_id
from carwatch_analysis.plotting import multi_paired_plot_auc, paired_plot_auc


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)

theme_kwargs = {"context": "talk", "style": "ticks", "palette": palette, "font_scale": 1.25}
sns.set_theme(**theme_kwargs)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

export = True

pg.options["round"] = 4

palette

## Setup Paths

In [ ]:
base_path = Path("../..")
export_path = base_path.joinpath("exports")
result_path = base_path.joinpath("results")
stats_path = result_path.joinpath("statistics")

img_path = result_path.joinpath("plots/graphical_abstract")

paper_path = Path(json.load(Path("../paper_path.json").open(encoding="utf-8"))["paper_path"])
paper_img_path = paper_path.joinpath("img/graphical_abstract")

bp.utils.file_handling.mkdirs([result_path, stats_path, img_path, paper_img_path])

## Load Data

### Cortisol Samples

In [ ]:
cort_path = export_path.joinpath("cortisol_samples_processed_all_reporting_types.csv")
cort_samples = load_cortisol_samples_reporting_times(cort_path)

cort_samples.head()

### Cortisol Features

In [ ]:
cort_features = load_long_format_csv(export_path.joinpath("cortisol_features_processed_all_reporting_types.csv"))
cort_features = create_unique_night_id(cort_features)
cort_features.head()

In [ ]:
# Don't consider IMU and IMU_App because the cortisol features are the same as
# Selfreport and App since only the wake onset differs, but not the sampling times
reporting_types_fine = [
    "Naive",
    "Selfreport without App",
    "Selfreport with App",
    "App",
    "Sensor + Selfreport without App",
    "Sensor + Selfreport with App",
    "Sensor + App",
]
reporting_types_coarse = ["Naive", "Selfreport", "App", "Sensor + Selfreport", "Sensor + App"]
delay_groups = ["None", "Short", "Moderate", "High"]

reporting_types_fine_rename = [s.replace("without", "w/o").replace("with", "w/") for s in reporting_types_fine]
rename_mapper_reporting_types = dict(zip(reporting_types_fine, reporting_types_fine_rename))

reporting_types_fine = reporting_types_fine_rename

cort_samples = cort_samples.rename(rename_mapper_reporting_types, level="reporting_type")
cort_features = cort_features.rename(rename_mapper_reporting_types, level="reporting_type")

## Data Selection

### Cortisol Samples

In [ ]:
cort_samples = multi_xs(cort_samples, reporting_types_fine, level="reporting_type")
cort_samples.head()

### Cortisol Features

In [ ]:
cort_features_analysis = cort_features.copy()
cort_features_analysis = multi_xs(cort_features_analysis, ["auc_g", "auc_i"], level="saliva_feature")
cort_features_analysis = multi_xs(cort_features_analysis, reporting_types_fine, level="reporting_type")
cort_features_analysis.head()

## Plots

### Paired Plots

#### $AUC_I$

In [ ]:
reporting_types = ["Naive", "Selfreport w/ App", "App"]

fig, axs = multi_paired_plot_auc(
    data=cort_features_analysis, saliva_feature="auc_i", reporting_types=reporting_types, figsize=(13, 5)
)

for path in [img_path, paper_img_path]:
    bp.utils.file_handling.export_figure(fig, "img_pairedplot_auci_abstract", path, ["pdf", "png"], dpi=300)